In [7]:
%reload_ext autoreload
%autoreload 2

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

import sys
sys.path.insert(0, './renderer')
sys.path.insert(0, '../')

from collections import namedtuple
import render_car_instances as rci
import utils.utils as uts
import glob
import os

import numpy as np
np.set_printoptions(threshold=np.nan)
import matplotlib.pyplot as plt
%matplotlib inline

import ntpath
pattern = '*%s*.%s' % ('_Camera_', 'jpg')

# Write pose maps to .npy files

In [8]:
codes_path = '/home/zhurui/Documents/mesh-voxelization/models/cars_64/codes.npy'
codes = np.load(codes_path)

print codes.shape
print np.amax(codes, axis=1)
print np.amin(codes, axis=1)

(10, 79)
[ 194.10701892   52.3540656    36.48827657   38.21597123   30.67944249
   28.11925775   20.46198426   19.39282606   14.72781895   15.88235606]
[-130.94684662  -48.79492285  -27.47333481  -33.49253033  -12.43573312
  -17.21450101  -13.24829926  -15.7550083   -12.31628475  -13.42541076]


# Clean combined dataset

In [ ]:
unique_lists = []
for split in ['train', 'val']:
    list_path = '/home/zhurui/Documents/deeplab/datasets/apolloscape/combined/train/split_dup/%s.txt'%split
    with open(list_path) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    print len(content)
    unique_content = list(set(content))
    print len(unique_content)
    
    unique_lists.append(unique_content)

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))
print len(intersection(unique_lists[0], unique_lists[1]))

final_lists = {}
final_lists['train'] = list(set(unique_lists[0]) - set(unique_lists[1]))
final_lists['val'] = unique_lists[1]
print len(final_lists['train']), len(final_lists['val'])

for split in ['train', 'val']:
    list_path = '/home/zhurui/Documents/deeplab/datasets/apolloscape/combined/train/split/%s.txt'%split
    thefile = open(list_path, 'w')
    for item in final_lists[split]:
      thefile.write("%s\n" % item)

In [ ]:
# Demo readling pkl and off files
import pickle as pkl
file_path = '/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/train/car_models/019-SUV.pkl'
with open(file_path) as f:
    a = pkl.load(f)
print a.keys()
# print a['vertices'].shape, a['faces'].shape
# print np.amax(a['vertices'], axis=0), np.amin(a['vertices'], axis=0)

off_path = '/home/zhurui/Documents/mesh-voxelization/models/cars/2_watertight/mc/019-SUV_ori_scale.off'
from off_utils import Mesh
off = Mesh.from_off(off_path)
print off.vertices.shape, off.faces.shape
print np.amax(off.vertices, axis=0), np.amin(off.vertices, axis=0)
print np.amax(off.faces), np.amin(off.faces)

# Batch processing

## For testing: process masks

In [20]:
import sys, os
sys.path.insert(0, './cocoapi/PythonAPI/pycocotools')
sys.path.insert(0, './cocoapi/PythonAPI')

print sys.path
from mask import decode
import pickle

det_masks_path = '../apolloscape/combined/test/det_masks'
det_masks_path = '../apolloscape/combined/test/pose_maps_02'
search_files = os.path.join(det_masks_path, pattern)
filenames = sorted(glob.glob(search_files))
print len(filenames)
for file_idx, filename in enumerate(filenames):
    print '--- %d/%d...'%(file_idx, len(filenames))
    with open(filename, 'rb') as f:
        mask = pickle.load(f)
        aa = decode(mask[1][1])

    print aa.shape

    for idx in range(aa.shape[2]):
        plt.imshow(aa[:, :, idx])
        plt.show()

    print mask[0][1].shape



['./cocoapi/PythonAPI', './cocoapi/PythonAPI/pycocotools', './cocoapi/PythonAPI', './cocoapi/PythonAPI/pycocotools', './cocoapi/PythonAPI', './cocoapi/PythonAPI/pycocotools', './cocoapi/PythonAPI', './cocoapi/PythonAPI/pycocotools', '/home/zhurui/anaconda2/lib/python2.7/site-packages', '../', '../', '../', '../', '../', '../', './renderer', '../', '../', '../', '../', '../', '/home/zhurui/anaconda2/lib/python2.7/site-packages', '../', '../', '../', '/home/zhurui/anaconda2/lib/python2.7/site-packages', '../', '/home/zhurui/anaconda2/lib/python2.7/site-packages', '../', './renderer', '', '/home/zhurui/.local/lib/python2.7/site-packages/mxnet-1.3.0-py2.7.egg', '/home/zhurui/.local/lib/python2.7/site-packages/graphviz-0.8.3-py2.7.egg', '/home/zhurui/.local/lib/python2.7/site-packages/requests-2.18.4-py2.7.egg', '/home/zhurui/.local/lib/python2.7/site-packages/certifi-2018.4.16-py2.7.egg', '/home/zhurui/.local/lib/python2.7/site-packages/urllib3-1.22-py2.7.egg', '/home/zhurui/.local/lib/pyt

ImportError: No module named mask

In [16]:
from PIL import Image
import h5py
import scipy.io as sio
import matplotlib.pyplot as plt
import utils.utils as uts
import pickle

%reload_ext autoreload
%autoreload 2
import render_car_instances as rci

sys.path[0] = '/home/zhurui/anaconda2/lib/python2.7/site-packages'
sys.path.insert(0, './renderer')

# splits = ['train']
splits = ['test']

for split in splits:
    Setting = namedtuple('Setting', ['data_dir'])
    setting = Setting('../apolloscape/combined/%s/'%split)

    visualizer = rci.CarPoseVisualizer(setting, scale=0.2, codes=codes)
    visualizer.load_car_models()
#     off_path = '/home/zhurui/Documents/mesh-voxelization/models/cars_64/2_watertight/'
#     visualizer.load_car_models_off(off_path)
    apollo_images_root = '/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/%s/images'%split
    print apollo_images_root
    search_files = os.path.join(apollo_images_root, pattern)
    filenames = sorted(glob.glob(search_files))
    print len(filenames)
    for file_idx, filename in enumerate(filenames):
        print '--- %d/%d...'%(file_idx, len(filenames))
        image_name = ntpath.basename(filename).replace('.jpg', '')
        folder_name = 'pose_maps_02'
        plot_path = filename.replace('images', folder_name).replace('.jpg', '_plot.jpg')
        
#         if os.path.isfile(filename.replace('images', folder_name).replace('.jpg', '_shape_id.png')):
# #             print 'Skipping '+filename
#             continue

        image_vis, seg_array, shape_id_map, depth, pose_map, image_rescaled, pose_list, shape_id_list = \
            visualizer.showAnn(image_name, if_visualize=True, if_save=False, plot_path=plot_path, is_training=False) 
        
        image_rescaled = Image.fromarray(np.uint8(image_rescaled))
        image_rescaled_name = filename.replace('images', folder_name).replace('.jpg', '_rescaled.png')
        image_rescaled.save(image_rescaled_name)

#         image_vis = Image.fromarray(np.uint8(image_vis))
#         image_vis_name = filename.replace('images', folder_name).replace('.jpg', '_vis.png')
#         image_vis.save(image_vis_name)

#         seg = Image.fromarray(np.uint8(seg_array))
#         seg_name = filename.replace('images', folder_name).replace('.jpg', '_seg.png')
#         seg.save(seg_name)
        
#         shape_id = Image.fromarray(np.uint8(shape_id_map))
#         shape_id_name = filename.replace('images', folder_name).replace('.jpg', '_shape_id.png')
#         shape_id.save(shape_id_name)

#         pose_dict_path = filename.replace('images', folder_name).replace('.jpg', '_posedict.npy')
#         pose_list_array = np.asarray(pose_list)
#         np.save(pose_dict_path, np.float32(pose_list_array)) 

#         pose_list, shape_id_list = visualizer.showId(image_name)

#         shape_id_dict_path = filename.replace('images', folder_name).replace('.jpg', '_shapeiddict.npy')
#         shape_id_list_array = np.asarray(shape_id_list)
#         np.save(shape_id_dict_path, np.float32(shape_id_list_array)) 

        if split == 'test':
            masks_root = '/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/%s/det_masks'%split
            mask_pkl = masks_root + '/%s.pkl'%image_name
            with open(mask_pkl, 'rb') as f:
                mask = pickle.load(f)
                print len(mask[0])
            

        break

INFO:root:loading 79 car models from ../apolloscape/combined/test//car_models...


/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/images
1041
--- 0/1041...
2


In [12]:
def crop_half(image):
#     print image.shape
    if len(image.shape)==3:
        return image[image.shape[0]//2:, :, :]
    elif len(image.shape)==2:
        return image[image.shape[0]//2:, :]

# splits = ['train']
splits = ['test']

for split in splits:
    Setting = namedtuple('Setting', ['data_dir'])
    setting = Setting('../apolloscape/combined/%s/'%split)

    visualizer = rci.CarPoseVisualizer(setting, scale=0.2)
    visualizer.load_car_models()
    apollo_images_root = '/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/%s/images'%split
    search_files = os.path.join(apollo_images_root, pattern)
    filenames = sorted(glob.glob(search_files))
    print len(filenames)
    for file_idx, filename in enumerate(filenames):
        print '--- %d/%d...'%(file_idx, len(filenames))
        image_name = ntpath.basename(filename).replace('.jpg', '')
        folder_name = 'pose_maps_02'
        plot_path = filename.replace('images', folder_name).replace('.jpg', '_plot.jpg')
        
#         for file_name in ['_rescaled.png', '_vis.png', '_seg.png', '_shape_id.png']: 
        for file_name in ['_rescaled.png']: 
            image_name = filename.replace('images', folder_name).replace('.jpg', file_name)
            
            if os.path.isfile(image_name.replace('.png', '_half.png')):
                print 'Skipping '+filename
                continue
            
            print image_name
            im = plt.imread(image_name)
            half_im = np.uint8(crop_half(im)*255.)
            Image.fromarray(half_im).save(image_name.replace('.png', '_half.png'))

            print 'Saved to '+image_name.replace('.png', '_half.png')

    #     break

INFO:root:loading 79 car models from ../apolloscape/combined/test//car_models...


1041
--- 0/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_034559609_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_034559609_Camera_5_rescaled_half.png
--- 1/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_034643831_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_034643831_Camera_5_rescaled_half.png
--- 2/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_035341773_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_model

(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_070620977_Camera_5_rescaled_half.png
--- 23/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_070723597_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_070723597_Camera_5_rescaled_half.png
--- 24/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_070821856_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_070821856_Camera_5_rescaled_half.png
--- 25/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_mo

(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_071810963_Camera_5_rescaled_half.png
--- 46/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_072008435_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_072008435_Camera_5_rescaled_half.png
--- 47/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_072106143_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_072106143_Camera_5_rescaled_half.png
--- 48/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_mo

(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_073852220_Camera_5_rescaled_half.png
--- 70/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_073855440_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_073855440_Camera_5_rescaled_half.png
--- 71/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_073857820_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/171206_073857820_Camera_5_rescaled_half.png
--- 72/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_mo

...

(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_022243867_Camera_5_rescaled_half.png
--- 935/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_022256829_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_022256829_Camera_5_rescaled_half.png
--- 936/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_022313630_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_022313630_Camera_5_rescaled_half.png
--- 937/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui

Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_023419933_Camera_5_rescaled_half.png
--- 958/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_023432241_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_023432241_Camera_5_rescaled_half.png
--- 959/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_023442080_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_023442080_Camera_5_rescaled_half.png
--- 960/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/

Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_024939597_Camera_5_rescaled_half.png
--- 982/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_024943777_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_024943777_Camera_5_rescaled_half.png
--- 983/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_025009879_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_025009879_Camera_5_rescaled_half.png
--- 984/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/

Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_030052921_Camera_5_rescaled_half.png
--- 1006/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_030057607_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_030057607_Camera_5_rescaled_half.png
--- 1007/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_030104139_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_030104139_Camera_5_rescaled_half.png
--- 1008/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitti

(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_032050919_Camera_5_rescaled_half.png
--- 1029/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_032349248_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_032349248_Camera_5_rescaled_half.png
--- 1030/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_032415287_Camera_5_rescaled.png
(544, 680, 3)
Saved to /home/zhurui/Documents/baidu/personal-code/car-fitting/rui_modelfitting/dataset-api/apolloscape/combined/test/pose_maps_02/180310_032415287_Camera_5_rescaled_half.png
--- 1031/1041...
/home/zhurui/Documents/baidu/personal-code/car-fitting/

In [ ]:
im = plt.imread(ignore_mask_path)
im.setflags(write=1)
print im.shape, im.dtype
plt.figure(figsize=(20, 20))
im[im>0]=255
plt.imshow(im)
plt.colorbar()
plt.show()
# uts.plot_images({'ignore_mask': im<255}, 
#                         layout=[1, 1], fig_size=5, save_fig=False, fig_name='ignore_mask')

In [ ]:
flat = im.flatten()
plt.hist(flat[(flat!=0) & (flat!=255)], bins=1000)
np.max(flat), np.min(flat), np.sum(np.logical_and(flat<255, flat>0))

In [ ]:
# Read .h5 files

# with h5py.File('aname-of-file.h5', 'r') as hf:
#     data = hf['name-of-dataset'][:]
print np.float32(pose_list_array)
print data.shape, data.dtype
import matplotlib.pyplot as plt
import utils.utils as uts
uts.plot_images({'pose_map': np.squeeze(np.uint8(seg_array))}, 
                layout=[1, 3], fig_size=40, save_fig=False, fig_name='pose_map')

# Read train/val splits

In [ ]:
splits = ['train', 'val']
split_files_folder = '../apolloscape/3d_car_instance_sample/split'

text_file = open('%s/%s.txt'%(split_files_folder, splits[0]), "r")
lines = [line for line in text_file.read().split('\n') if '.jpg' in line]

print len(lines), lines

# Obsolete

In [ ]:
print pose_map.shape
import numpy as np
np.set_printoptions(threshold=np.nan)

import matplotlib.pyplot as plt
import utils.utils as uts
uts.plot_images({'pose_map': np.squeeze(pose_map[:, :, 0])}, 
                layout=[1, 3], fig_size=40, save_fig=False, fig_name='pose_map')

In [ ]:
aa = pose_map.flatten().tolist()
bb = [np.float16(it) for it in pose_map.flatten()]
print len(aa), len(bb), pose_map.shape